### Постановка задачи
Построить модель линейной регрессии энергопотребления здания, используя температуру воздуха (air_temperature) и влажность (dew_temperature).

Рассчитать качество построенной модели по проверочным данным.

Данные:
* http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/train.0.0.csv.gz
Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/

© ITtensive, 2020

### Подключение библиотек

In [5]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

### Загрузка данных

In [6]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
energy_0 = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.0.csv.gz")
print (energy_0.info()) #Посмотрим сводную информацию методом info():
#мы узнаем, что в таблице 8784 строки (от 0 до 8783), 
# 4 столбца - количество ненулевых значений (non-null), тип данных: 
#  int64 - целые числа (числа размером 8 байт);
# object - абстракция Python для данных,
# float64 – число с плавающей запятой (числа размером 8 байт).

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   building_id    8784 non-null   int64  
 1   meter          8784 non-null   int64  
 2   timestamp      8784 non-null   object 
 3   meter_reading  8784 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 240.2+ KB
None


### Объединение данных и фильтрация

In [7]:
# объединяем в таблицу, с помощью параметра how="left" мы указали, что главным #является датафрейм слева, название столбца – «building_id».
energy_0 = pd.merge(left=energy_0, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
# проиндексируем в таблице данных energy_0 по столбцам "timestamp", "site_id"
energy_0.set_index(["timestamp", "site_id"], inplace=True)
# проиндексируем в таблице данных weather по столбцам "timestamp", "site_id"
weather.set_index(["timestamp", "site_id"], inplace=True)
# объединяем в таблицу: слева данные energy_0, справа данные weather
energy_0 = pd.merge(left=energy_0, right=weather, how="left",
                   left_index=True, right_index=True)
# сбросим индекс по данным  energy_0
energy_0.reset_index(inplace=True)
# отфильтруем только положительные значения 
energy_0 = energy_0[energy_0["meter_reading"] > 0]
# преобразовываем формат времени в столбце timestamp
energy_0["timestamp"] = pd.to_datetime(energy_0["timestamp"])
# атрибут dt.hour возвращает массив данных  timestamp к базовым данным (2,4…)
energy_0["hour"] = energy_0["timestamp"].dt.hour
# просмотрим первые строки
print (energy_0.head())

               timestamp  site_id  building_id  meter  meter_reading  \
704  2016-01-30 08:00:00        0            0      0        43.6839   
725  2016-01-31 05:00:00        0            0      0        37.5408   
737  2016-01-31 17:00:00        0            0      0        52.5571   
2366 2016-04-08 14:00:00        0            0      0        59.3827   
2923 2016-05-01 19:00:00        0            0      0       448.0000   

     primary_use  square_feet  year_built  floor_count  air_temperature  \
704    Education         7432      2008.0          NaN              8.3   
725    Education         7432      2008.0          NaN             12.8   
737    Education         7432      2008.0          NaN             20.6   
2366   Education         7432      2008.0          NaN             21.7   
2923   Education         7432      2008.0          NaN             31.1   

      cloud_coverage  dew_temperature  precip_depth_1_hr  sea_level_pressure  \
704              NaN              6.

### Добавление часа в данные

In [8]:
energy_0["timestamp"] = pd.to_datetime(energy_0["timestamp"])
energy_0["hour"] = energy_0["timestamp"].dt.hour

### Разделение данных на обучение и проверку

In [9]:
# Выделим 20% всех данных на проверку, остальные оставим на обучение
# используя модуль train_test_split разделяем данные для машинного обучения.
# переменная test_size фактически указывает пропорцию набора тестов – 20%.
energy_0_train, energy_0_test = train_test_split(energy_0, test_size=0.2)
print (energy_0_train.head())


               timestamp  site_id  building_id  meter  meter_reading  \
7772 2016-11-19 20:00:00        0            0      0        211.594   
5810 2016-08-30 02:00:00        0            0      0        247.087   
7876 2016-11-24 04:00:00        0            0      0        161.084   
6647 2016-10-03 23:00:00        0            0      0        254.595   
7318 2016-10-31 22:00:00        0            0      0        205.451   

     primary_use  square_feet  year_built  floor_count  air_temperature  \
7772   Education         7432      2008.0          NaN             25.0   
5810   Education         7432      2008.0          NaN             24.4   
7876   Education         7432      2008.0          NaN             18.9   
6647   Education         7432      2008.0          NaN             26.7   
7318   Education         7432      2008.0          NaN             26.1   

      cloud_coverage  dew_temperature  precip_depth_1_hr  sea_level_pressure  \
7772             4.0             12.

### Модель линейной регрессии и среднее
meter_reading = A * air_temperature + B * dew_temperature + C

Дополнительно вычислим среднее по часам, чтобы сравнить линейную регрессию с более простой моделью

In [12]:
# чтобы сравнить линейную регрессию с более простой моделью
# дополнительно вычислим среднее по часам
energy_0_train_averages = energy_0_train.groupby("hour").mean()["meter_reading"]
energy_0_train_lr = pd.DataFrame(energy_0_train,
    columns=["meter_reading", "air_temperature", "dew_temperature"])
y = energy_0_train_lr["meter_reading"]
x = energy_0_train_lr.drop(labels=["meter_reading"], axis=1)
model = LinearRegression().fit(x, y)
print (model.coef_, model.intercept_)
#полученные коэффициенты будем использовать для оценки модели:

[2.27425656 4.06642286] 101.75007175411153


### Оценка модели

In [13]:
def calculate_model (x):
    meter_reading_log = np.log(x.meter_reading + 1)
    meter_reading_mean = np.log(energy_0_train_averages[x.hour] + 1)
    meter_reading_lr = np.log(1 + x.air_temperature * model.coef_[0] + 
                                  x.dew_temperature * model.coef_[1] +
                                  model.intercept_)
    x["meter_reading_lr_q"] = (meter_reading_log - meter_reading_lr)**2
    x["meter_reading_mean_q"] = (meter_reading_log - meter_reading_mean)**2
    return x

energy_0_test = energy_0_test.apply(calculate_model,
                                    axis=1, result_type="expand")
energy_0_test_lr_rmsle = np.sqrt(energy_0_test["meter_reading_lr_q"].sum() / len(energy_0_test))
energy_0_test_mean_rmsle = np.sqrt(energy_0_test["meter_reading_mean_q"].sum() / len(energy_0_test))
print ("Качество среднего:", energy_0_test_mean_rmsle)
print ("Качество линейной регрессии:", energy_0_test_lr_rmsle)

Качество среднего: 0.2581035097558345
Качество линейной регрессии: 0.22297486656537502
